In [7]:

import jsonlines
import pandas as pd
import warnings
import numpy as np


In [ ]:
date_limits = ['2019-03-01 00:00:00', 
               '2019-06-01 00:00:00', 
               '2019-09-01 00:00:00', 
               '2020-01-01 00:00:00', 
               '2020-03-01 00:00:00', 
               '2020-06-01 00:00:00', 
               '2020-09-01 00:00:00', 
               '2021-01-01 00:00:00', 
               '2021-03-01 00:00:00', 
               '2021-06-01 00:00:00'
            ]

COUNT = len(date_limits)

In [8]:
def dfToCSV(posts, filename):
    posts.to_csv(filename, index=False)

def postsFromNDJSON(filename):
    posts = []
    with jsonlines.open(filename) as reader:
        for line in reader:
            line = line["Item"]
            posts.append([line["created"]["S"], line["title"]["S"], line["id"]["S"]])
    posts = pd.DataFrame(posts,columns=['created', 'title', 'id'])

    return posts

def sortByTime(data):
    data = data.sort_values(by='created')
    return data

posts = postsFromNDJSON("worldnews.jsonl")
posts = sortByTime(posts)
posts.drop(posts.loc[posts['title'] == 'None'].index, inplace=True)
dfToCSV(posts=posts, filename="worldnews.csv")
print(posts)


                    created  \
360671  2018-12-31 02:25:14   
270512  2018-12-31 02:27:21   
112640  2018-12-31 02:27:56   
294115  2018-12-31 02:29:05   
214742  2018-12-31 02:29:18   
...                     ...   
498320  2021-04-09 21:26:18   
427024  2021-04-09 21:26:19   
816977  2021-04-09 21:26:20   
144817  2021-04-09 21:26:21   
516521  2021-04-09 21:26:28   

                                                    title      id  
360671  پسکوف: هیچ اقدامی علیه مظنونان معرفی گردیده در...  ab2og2  
270512  One of the companies contracted by the governm...  ab2p2z  
112640  Donald Trump: Outgoing chief of staff John Kel...  ab2p8u  
294115  Todd Bowles Fired as Jets Head Coach After 4 S...  ab2pl2  
214742  Macron's former aide admits using diplomatic p...  ab2pn3  
...                                                   ...     ...  
498320  Prince Charles Visits Queen Elizabeth II After...  mnr2wp  
427024  Job postings hint at winners of NYC and London...  mnr2x1  
816977  Tell Us

In [ ]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    lemmatizer = WordNetLemmatizer()

    # Tokenize words
    words = word_tokenize(text)

    # Remove stopwords and lemmatize words
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]

    words_split = []
    for word in words:
        words_split.extend(re.split('(\d+)',word))

    return ' '.join(words_split)

def isEnglish(text):
    try:
        text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True


def separateNumbersAlphabets(str):
    numbers = re.findall(r'[0-9]+', str)
    alphabets = re.findall(r'[a-zA-Z]+', str)
    print(*numbers)
    print(*alphabets)

posts = postsFromCSV(infile)

start = datetime.datetime.now()

lemmed = []

for idx in posts.index:
    if idx % 50000 == 0:
        print(idx / 1000)
    text = posts['title'][idx]

    text = preprocess(text).replace('http', '')

    if isEnglish(text):
        lemmed.append(text)
    else:
        lemmed.append('DELETEME')

stop = datetime.datetime.now()
print("Took", stop - start, "seconds")

posts["title"] = lemmed

posts.drop(posts.loc[posts['title'] == 'DELETEME'].index, inplace=True)
posts.drop(posts.loc[posts['title'] == ''].index, inplace=True)

print(posts)
postsToCSV(posts, outfile)